# Kickstarter_survivors
### Sifan Liu

## Set up and load data

In [1]:
# Load packages and set up
import numpy as np
import pandas as pd
import json
import glob

def CustomParser(data):
    import json
    try:
        j1 = json.loads(data)
        return j1
    except ValueError:
        pass

JSONconverters = {'location':CustomParser, 'category':CustomParser, 'creator':CustomParser}


In [3]:
# read data from csv

def ReadAllfiles(path):
    all_files = glob.glob(path + "/*.csv")
    df_from_each_file = (pd.read_csv(f,converters = JSONconverters) for f in all_files)
    df = pd.concat(df_from_each_file, ignore_index = True)
    return df

path2018 = r'../datasample/Kickstarter_2018-02-15T03_20_44_743Z'
# path2017 = r'source/Kickstarter_2017-02-15T22_22_48_377Z'
# path2015 = r'source/Kickstarter_2015-12-17T12_09_06_107Z'

# df2015 = ReadAllfiles(path2015)
# df2017 = ReadAllfiles(path2017)
df2018 = ReadAllfiles(path2018)

# df = pd.concat([df2018, df2017, df2015]).drop_duplicates(["id"]).reset_index(drop=True)


TypeError: ("unhashable type: 'dict'", 'occurred at index category')

In [18]:
df = df2018.drop_duplicates(['launched_at'])
df.info()
# sample_data ------------------------------------------
# df = pd.read_csv('source/kickstarter_sample.csv', converters = JSONconverters)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177647 entries, 0 to 195613
Data columns (total 37 columns):
backers_count               177647 non-null int64
blurb                       177637 non-null object
category                    177647 non-null object
converted_pledged_amount    177647 non-null int64
country                     177647 non-null object
created_at                  177647 non-null int64
creator                     177647 non-null object
currency                    177647 non-null object
currency_symbol             177647 non-null object
currency_trailing_code      177647 non-null bool
current_currency            177647 non-null object
deadline                    177647 non-null int64
disable_communication       177647 non-null bool
fx_rate                     177647 non-null float64
goal                        177647 non-null float64
id                          177647 non-null int64
is_starrable                177647 non-null bool
launched_at                 177

In [19]:
# extract json information to columns
def JsonConcate(dataframe, column):
    temp = dataframe[column].apply(pd.Series)
    temp = temp.rename(columns = lambda x : column + '_' + str(x))
    dataframe = pd.concat([dataframe[:],temp[:]],axis =1)
    return dataframe

for col in ('location','category','creator'):
    df = JsonConcate(df,col)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177647 entries, 0 to 195613
Data columns (total 62 columns):
backers_count                177647 non-null int64
blurb                        177637 non-null object
category                     177647 non-null object
converted_pledged_amount     177647 non-null int64
country                      177647 non-null object
created_at                   177647 non-null int64
creator                      177647 non-null object
currency                     177647 non-null object
currency_symbol              177647 non-null object
currency_trailing_code       177647 non-null bool
current_currency             177647 non-null object
deadline                     177647 non-null int64
disable_communication        177647 non-null bool
fx_rate                      177647 non-null float64
goal                         177647 non-null float64
id                           177647 non-null int64
is_starrable                 177647 non-null bool
launched_at   

In [20]:
place2msa = pd.read_csv("source/place2msa.csv", encoding = 'latin-1')
# remove 'city'|'town'|'CDP'
place2msa['place'] = place2msa['PLACENAME'].str.replace(r'(\s\w+)$','')

place2msa.sample(5)

,Unnamed: 0,STATE,STATEFP,PLACEFP,PLACENAME,TYPE,FUNCSTAT,cty_name,cty_alt,FIPS_City,...,cbsa_name,metro_micro,csa_name,st_name,countyFIPS,central_outlying,fips,top100,Frey52,place
13472,13473,MI,26,27880,Ferndale city,Incorporated Place,A,Oakland County,NaN,2627880,...,"Detroit-Warren-Dearborn, MI",Metropolitan Statistical Area,"Detroit-Warren-Ann Arbor, MI",Michigan,125.0,Central,26125.0,1.0,1.0,Ferndale
34154,34155,TN,47,64440,Rockwood city,Incorporated Place,A,Roane County,NaN,4764440,...,"Knoxville, TN",Metropolitan Statistical Area,"Knoxville-Morristown-Sevierville, TN",Tennessee,145.0,Outlying,47145.0,1.0,0.0,Rockwood
7997,7998,IN,18,51732,Mount Vernon city,Incorporated Place,A,Posey County,NaN,1851732,...,"Evansville, IN-KY",Metropolitan Statistical Area,NaN,Indiana,129.0,Outlying,18129.0,0.0,0.0,Mount Vernon
11497,11498,ME,23,73472,Square Lake UT,County Subdivision,S,Aroostook County,NaN,2373472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Square Lake
22086,22087,NJ,34,51360,New Egypt CDP,Census Designated Place,S,Ocean County,NaN,3451360,...,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,"New York-Newark, NY-NJ-CT-PA",New Jersey,29.0,Central,34029.0,1.0,1.0,New Egypt


In [ ]:
## TODO

# merge population data

## Clean Kickstarter dataset

In [21]:
# convert unix time
time_cols = ['created_at', 'deadline', 'state_changed_at','launched_at']
df[time_cols] = df[time_cols].apply(pd.to_datetime,unit='s')

In [22]:
# calculate duration
df['life'] = df['deadline'] - df['launched_at']
df['duration'] = df['state_changed_at'] - df['launched_at']
df['prep'] =  df['launched_at'] - df['created_at']

# df['state_changed_at'][0].year

In [23]:
# factorize project state
df['state'] = df['state'].astype('category')
pd.pivot_table(df, index = ['state'])

,backers_count,category_color,category_id,category_parent_id,category_position,converted_pledged_amount,creator_id,creator_is_registered,currency_trailing_code,disable_communication,fx_rate,goal,id,is_starrable,location_id,pledged,spotlight,staff_pick,static_usd_rate,usd_pledged
state,,,,,,,,,,,,,,,,,,,,
canceled,14.595892,1.127758e+07,251.300840,12.062188,6.531548,1743.495574,1.078063e+09,1.0,0.819110,0.0,1.016370,129421.320380,1.081303e+09,0.00000,3.406730e+06,2071.816492,0.0,0.034158,1.018427,1745.585310
failed,13.024047,1.180186e+07,216.307880,11.858619,6.932738,1144.908184,1.074044e+09,1.0,0.843950,0.0,1.017990,88373.330867,1.077541e+09,0.00000,3.493788e+06,1357.625354,0.0,0.035118,1.022837,1146.593782
live,80.914510,1.075089e+07,136.398021,11.306412,6.444191,6546.533486,1.062190e+09,1.0,0.762303,0.0,0.992477,46928.084221,1.068464e+09,0.93861,3.686239e+06,17144.056481,0.0,0.065195,1.000137,6660.775202
successful,235.938560,1.225656e+07,113.349221,11.649377,7.542104,19338.581673,1.073328e+09,1.0,0.895925,0.0,1.017314,9967.393967,1.073053e+09,0.00000,3.293887e+06,20730.996588,1.0,0.211469,1.021316,19340.536221
suspended,25.078203,9.734057e+06,281.163062,12.978369,5.725458,3818.810316,1.090515e+09,1.0,0.795341,1.0,1.018514,248081.559068,1.107546e+09,0.00000,3.151386e+06,4122.225191,0.0,0.000000,1.018617,3804.140207


In [24]:
df['category_broad'] = df['category_slug'].str.extract(r'([^\/]+)', expand = False).astype('category')
df['category_broad'].describe()

count     177647
unique        15
top        music
freq       24697
Name: category_broad, dtype: object

In [25]:
# generate gender by first name
# STEP 1: strip first name
df['first_name'] = df['creator_name'].str.extract(r'([^\s]+)',expand = False).str.title()


# STEP 2: gender
# https://pypi.org/project/gender-guesser/
import gender_guesser.detector as gender
from gender_detector import GenderDetector
d = gender.Detector()
g = GenderDetector('us')

# STEP 3: apply
df['gender'] = df['first_name'].apply(d.get_gender)
df['Gender'] = df['first_name'].apply(g.getGender)

pd.crosstab(df['gender'], df['Gender'])

ModuleNotFoundError: No module named 'gender_guesser'

In [48]:
# match place to MSA

df['location_place'] = df['location_short_name'].str.extract(r'([^\,]+)',expand = False)

merge = pd.merge(df, place2msa, how = 'left',
                  left_on=['location_state','location_place'],
                  right_on=['STATE','place'])



In [54]:
# check for the unmatched places

temp = merge[merge['top100'].isnull()]
temp = temp[temp['location_country'] == "US"]

temp.groupby(['location_localized_name','location_short_name'])['goal'].count().nlargest(50)



location_localized_name  location_short_name 
Nashville                Nashville, TN           1393
Indianapolis             Indianapolis, IN         545
Hollywood                Hollywood, CA            434
Louisville               Louisville, KY           317
Boise                    Boise, ID                281
Honolulu                 Honolulu, HI             277
Black Rock City          Black Rock City, NV      253
Long Island              Long Island, NY          248
Lexington                Lexington, KY            241
Lower East Side          Lower East Side, NY      187
Athens                   Athens, GA               149
South Florida            South Florida, FL        143
North Hollywood          North Hollywood, CA      105
Venice                   Venice, CA                92
East Village             East Village, NY          85
Brattleboro              Brattleboro, VT           76
Ventura                  Ventura, CA               76
Johnson                  Johnson, VT

In [55]:
# df = df.drop_duplicates(["id"])
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11069 entries, 4 to 201978
Data columns (total 89 columns):
backers_count                11069 non-null int64
blurb                        11068 non-null object
category                     11069 non-null object
converted_pledged_amount     11069 non-null int64
country                      11069 non-null object
created_at                   11069 non-null datetime64[ns]
creator                      11069 non-null object
currency                     11069 non-null object
currency_symbol              11069 non-null object
currency_trailing_code       11069 non-null bool
current_currency             11069 non-null object
deadline                     11069 non-null datetime64[ns]
disable_communication        11069 non-null bool
fx_rate                      11069 non-null float64
goal                         11069 non-null float64
id                           11069 non-null int64
is_starrable                 11069 non-null bool
launched_at   

## Summary: selected columns

In [70]:
# keep selected columns
col = ['location_country','location_state','location_localized_name','location_name','cbsa_name','top100',
       'backers_count','goal','pledged','creator_name','gender','category_broad','category_name',
       'launched_at','deadline','state_changed_at','state','life','duration','prep']

df_sample = df[col]

# table = pd.pivot_table(df_sample, index=["location_state","location_country"], aggfunc = [np.mean,len])
# table.query('location_country == ["US"]')

In [3]:
# df = pd.read_pickle("master")

In [74]:
# df_sample.to_pickle("sample")
df_sample.to_pickle("master")

In [71]:
df['state'].value_counts()

successful    105055
failed         82220
canceled        9764
live            4358
suspended        678
Name: state, dtype: int64

In [72]:
df.isnull().sum()

backers_count                    0
blurb                           11
category                         0
converted_pledged_amount         0
country                          0
created_at                       0
creator                          0
currency                         0
currency_symbol                  0
currency_trailing_code           0
current_currency                 0
deadline                         0
disable_communication            0
fx_rate                          0
goal                             0
id                               0
is_starrable                     0
launched_at                      0
name                             1
photo                            0
pledged                          0
profile                          0
slug                             0
source_url                       0
spotlight                        0
staff_pick                       0
state                            0
state_changed_at                 0
static_usd_rate     

In [73]:
df.shape

(202075, 89)